In [20]:
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
import statsmodels.api as sm
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

In [21]:
df = pd.read_csv('source/FReDA4.csv')
# df.info()

In [22]:
df2 = df.copy().dropna()
# df2.info()

In [23]:
df2 = (
    df2.groupby("CoupleId")
    .filter(lambda g: {"Anchor", "Partner"}.issubset(set(g["Role"])))
)

In [24]:
touch_factors = [
    'Frequency of Touch',
    'Wish for Touch'
]

individual_factors = [
    'Sex',
    'Age',
    'Work Status',
    # 'East',
    # 'Urbanization',
]

personality_factors = [
    'Extraversion',
    'Agreeableness',
    'Conscientiousness',
    'Openness',
    'Neuroticism',
    # 'Conservatism',
    # 'Religiosity',
]

wellbeing_factors = [
    'Depressiveness',
    'Loneliness',
    'Self-esteem',
    'Life Satisfaction',
    'Health',
]

relationship_factors = [
    'Relationship Sex',
    'Relationship Length',
    'Age difference',
    'Married',
    'Cohabitation',
    'Kids',
    'Communication Quality',
    'Relationship Satisfaction',
    'Conflict Management',
]

all_factors = [
    'Sex',
    'Age',
    'Work Status',
    # 'East',
    # 'Urbanization',
    'Extraversion',
    'Agreeableness',
    'Conscientiousness',
    'Openness',
    'Neuroticism',
    # 'Conservatism',
    # 'Religiosity',
    'Depressiveness',
    'Loneliness',
    'Self-esteem',
    'Life Satisfaction',
    'Health',
    'Relationship Sex',
    'Relationship Length',
    'Age difference',
    'Married',
    'Cohabitation',
    'Kids',
    'Communication Quality',
    'Relationship Satisfaction',
    'Conflict Management',
]

In [25]:
X = df2[all_factors]
y = df2["Frequency of Touch"]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X = pd.DataFrame(X_scaled, columns=all_factors, index=X.index)

In [26]:
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import numpy as np

kf = KFold(n_splits=4, shuffle=True)

cv_scores = []

for train_index, test_index in kf.split(X):
    # Split the data into training and test sets
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Fit a linear regression model
    model = LinearRegression().fit(X_train, y_train)

    # Make predictions on the test set
    y_pred = model.predict(X_test)

    # Calculate the R^2 score
    score = r2_score(y_test, y_pred)

    # Append the score to the list of cross-validation scores
    cv_scores.append(score)

# Calculate the mean and standard deviation of the cross-validation scores
mean_score = np.mean(cv_scores)
std_score = np.std(cv_scores)

print(f"Cross-validation scores: {cv_scores}")
print(f"Mean R^2 score: {mean_score:.3f}")
print(f"Standard deviation of R^2 scores: {std_score:.3f}")

Cross-validation scores: [0.3420737884691004, 0.3355829716525759, 0.32959985673782566, 0.33130638415695035]
Mean R^2 score: 0.335
Standard deviation of R^2 scores: 0.005


In [27]:
def forward_selection(X, y, threshold_in=0.05):
    initial_features = X.columns.tolist()
    best_features = []

    while len(initial_features) > 0:
        remaining_features = list(set(initial_features) - set(best_features))
        new_pval = pd.Series(index=remaining_features)

        for feature in remaining_features:
            model = sm.OLS(y, sm.add_constant(X[best_features + [feature]])).fit()
            new_pval[feature] = model.pvalues[feature]

        min_p_value = new_pval.min()

        if min_p_value < threshold_in:
            best_features.append(new_pval.idxmin())
        else:
            break

    return best_features

forward_selected_features = forward_selection(X, y)
print("Selected features using forward selection: ", forward_selected_features)

Selected features using forward selection:  ['Communication Quality', 'Relationship Satisfaction', 'Relationship Length', 'Kids', 'Cohabitation', 'Work Status', 'Age', 'Agreeableness', 'Depressiveness', 'Loneliness', 'Extraversion', 'Conscientiousness', 'Married', 'Openness', 'Neuroticism', 'Self-esteem']


In [28]:
def stepwise_selection(X, y,
                       threshold_in=0.15,
                       threshold_out=0.15):

    included = []
    while True:
        changed = False
        # ---------- FORWARD STEP ----------
        excluded = list(set(X.columns) - set(included))
        new_pvals = pd.Series(index=excluded, dtype=float)

        for feature in excluded:
            model = sm.OLS(y, sm.add_constant(X[included + [feature]])).fit()
            new_pvals[feature] = model.pvalues[feature]

        if not new_pvals.empty:
            best_feature = new_pvals.idxmin()
            best_pval = new_pvals.min()

            if best_pval < threshold_in:
                included.append(best_feature)
                changed = True

        # ---------- BACKWARD STEP ----------
        if included:
            model = sm.OLS(y, sm.add_constant(X[included])).fit()
            pvals = model.pvalues.iloc[1:]  # drop the intercept

            worst_feature = pvals.idxmax()
            worst_pval = pvals.max()

            if worst_pval > threshold_out:
                included.remove(worst_feature)
                changed = True

        if not changed:
            break

    return included

bidirectional_selected_features = forward_selection(X, y)
print("Selected features using Bidirectional Elimination: ", bidirectional_selected_features)

Selected features using Bidirectional Elimination:  ['Communication Quality', 'Relationship Satisfaction', 'Relationship Length', 'Kids', 'Cohabitation', 'Work Status', 'Age', 'Agreeableness', 'Depressiveness', 'Loneliness', 'Extraversion', 'Conscientiousness', 'Married', 'Openness', 'Neuroticism', 'Self-esteem']


In [29]:
final_model = sm.OLS(y, sm.add_constant(X[bidirectional_selected_features])).fit()
print(final_model.summary())

                            OLS Regression Results                            
Dep. Variable:     Frequency of Touch   R-squared:                       0.339
Model:                            OLS   Adj. R-squared:                  0.338
Method:                 Least Squares   F-statistic:                     275.6
Date:                Tue, 25 Nov 2025   Prob (F-statistic):               0.00
Time:                        16:08:44   Log-Likelihood:                -12506.
No. Observations:                8620   AIC:                         2.505e+04
Df Residuals:                    8603   BIC:                         2.517e+04
Df Model:                          16                                         
Covariance Type:            nonrobust                                         
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
const                 